In [1]:
! pip install -U langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

In [3]:
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

In [8]:
user_data = {
    "tax_identifier": None,
    "surname": None,
    "first_name": None,
    "date_of_birth": None,
    "home_address": None,
    "declaration_office": None,
    "purchase_date": None,
    "car_market_value": None
}

In [53]:
@tool
def get_user_info(name: str = None, surname: str = None, PESEL: str = None):
    """If any personal data detected in query use this tool"""
    
    # Dictionary with default empty values
    user_data = {
        "tax_identifier": "",
        "surname": "",
        "first_name": "",
        "date_of_birth": "",
        "home_address": "",
        "declaration_office": "",
        "purchase_date": "",
        "car_market_value": ""
    }

    # Update the dictionary with provided values (if they are not None)
    
    if name:
        user_data["first_name"] = name
    '''
    if tax_identifier:
        user_data["tax_identifier"] = tax_identifier
    if surname:
        user_data["surname"] = surname    
    if date_of_birth:
        user_data["date_of_birth"] = date_of_birth
    if home_address:
        user_data["home_address"] = home_address
    if declaration_office:
        user_data["declaration_office"] = declaration_office
    if purchase_date:
        user_data["purchase_date"] = purchase_date
    if car_market_value:
        user_data["car_market_value"] = car_market_value
    '''
        
    return user_data


In [54]:
'''
@tool
def get_user_info(name: str = None, surname: str = None, pesel: str = None, tax_identifier: str = None, date_of_birth: str = None, home_address: str = None, declaration_office: str = None, purchase_date: str = None, car_market_value: str = None):
    """If any personal data detected in query use this tool"""
    
    # Dictionary with default empty values
    user_data = {
        "tax_identifier": "",
        "surname": "",
        "first_name": "",
        "date_of_birth": "",
        "home_address": "",
        "declaration_office": "",
        "purchase_date": "",
        "car_market_value": ""
    }

    # Update the dictionary with provided values (if they are not None)
    if tax_identifier:
        user_data["tax_identifier"] = tax_identifier
    if surname:
        user_data["surname"] = surname
    if name:
        user_data["first_name"] = name
    if date_of_birth:
        user_data["date_of_birth"] = date_of_birth
    if home_address:
        user_data["home_address"] = home_address
    if declaration_office:
        user_data["declaration_office"] = declaration_office
    if purchase_date:
        user_data["purchase_date"] = purchase_date
    if car_market_value:
        user_data["car_market_value"] = car_market_value

    return user_data
'''

'\n@tool\ndef get_user_info(name: str = None, surname: str = None, pesel: str = None, tax_identifier: str = None, date_of_birth: str = None, home_address: str = None, declaration_office: str = None, purchase_date: str = None, car_market_value: str = None):\n    """If any personal data detected in query use this tool"""\n    \n    # Dictionary with default empty values\n    user_data = {\n        "tax_identifier": "",\n        "surname": "",\n        "first_name": "",\n        "date_of_birth": "",\n        "home_address": "",\n        "declaration_office": "",\n        "purchase_date": "",\n        "car_market_value": ""\n    }\n\n    # Update the dictionary with provided values (if they are not None)\n    if tax_identifier:\n        user_data["tax_identifier"] = tax_identifier\n    if surname:\n        user_data["surname"] = surname\n    if name:\n        user_data["first_name"] = name\n    if date_of_birth:\n        user_data["date_of_birth"] = date_of_birth\n    if home_address:\n   

In [55]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."
tool_node

@tool
def get_coolest_cities():
    """Get a list of coolest cities"""
    return "nyc, sf"

In [56]:
tools = [get_weather, get_coolest_cities, get_user_info]
tool_node = ToolNode(tools)

In [60]:
from typing import Literal
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode


model_with_tools = ChatOpenAI(model="gpt-4").bind_tools(tools)
model_with_tools.invoke("Which cities are cool").tool_calls

[{'name': 'get_coolest_cities',
  'args': {},
  'id': 'call_xh7GHEJUgToSoEFz0tCBueZ6',
  'type': 'tool_call'}]

In [58]:
tool_node.invoke({"messages": [model_with_tools.invoke("what's the weather in sf?")]})

{'messages': [ToolMessage(content="It's 60 degrees and foggy.", name='get_weather', tool_call_id='call_ZGfvgCrx2KEYxQVJxX1S1Olq')]}